# Multi-Perceptron Grid_Search

## Table of Contents

- [Format All Results](#Format-All-Results)


In [101]:
import polars as pl
from icecream import ic
import pandas as pd
from great_tables import GT, md, html, from_column, style, loc
from assignment_3_tools import parquet_to_dict

## Format All Results

In [102]:
result_path = '../../Data/GoogleDrive/MLP_Results/'

result_pl = {
    key:value.collect().drop('Dataset_Name')
    for key, value in parquet_to_dict(result_path).items()
}

result_pl = {
    key: value if key.endswith('baseline') else value.with_columns(pl.lit("").alias("Grid_Variable"))
    for key, value in result_pl.items()
}

for key, frame in result_pl.items():
    if key.endswith('baseline'):
        continue
    else:
        result_dict = frame.to_dict(as_series=False)
        params = []
        for entry in result_dict['Parameters']:
            param_values = "".join([str(value) for value in entry.values()])
            params.append(param_values)
        result_dict['Parameters'] = params
        result_pl[key] = pl.DataFrame(result_dict)


param_name = dict()
unique_name = list()
for key in result_pl:
    if not key.endswith('baseline'):
        unique_name.append(key[4:])
unique_name = set(unique_name)

for key in unique_name:
        prefix, name = key.split('-', 1)
        param_name.setdefault(name, []).append(key)

result_table = dict()
result_combo = dict()
for param, name_list in param_name.items():
    for name in name_list:
        prefixes = ['test', 'grid']
        for prefix in prefixes:
            alt = f"{prefix}{name}"
            baseline = f"{prefix}_results-baseline"
            alt_df = result_pl[alt].with_columns(pl.col("Parameters").cast(pl.Utf8))
            baseline_dict = result_pl[baseline].to_dict(as_series=False)
            baseline_dict['Parameters'] = [val for test, val in baseline_dict['Parameters'][0].items() if test == param][0]
            baseline_df = pl.DataFrame(baseline_dict).with_columns(pl.col("Parameters").cast(pl.Utf8))
            combo = pl.concat([baseline_df, alt_df])
            result_combo[alt] = combo
            if prefix == "test":
                table = (
                    GT(combo.sort(by='Recall', descending=True))
                    .tab_header(
                        title=f"{param.title().replace('_', ' ')}",
                        subtitle=f"Test Set Prediction Results"
                    )
                    .tab_spanner(
                        label="Metrics",
                        columns=["Recall", "ROC_AUC", "Accuracy", "Fit_Time"]
                    )
                    .cols_label(
                        Grid_Variable=html(""),
                        Parameters=html(f"{param.title()} <br> Parameter"),
                        ROC_AUC=html("ROC AUC"),
                        Fit_Time=html("Fit Time (s)"),
                    )
                    .fmt_number(columns=["Recall","ROC_AUC","Accuracy"], decimals=2)
                    .fmt_number(columns=["Fit_Time"], decimals=0)
                )
            elif prefix == "grid":
                table = (
                    GT(combo.sort(by='Recall', descending=True))
                    .tab_header(
                        title=f"{param.title().replace('_', ' ')}",
                        subtitle=f"Cross Validation Results"
                    )
                    .fmt_number(columns=["Recall"], decimals=2)
                    .fmt_number(columns=["Fit_Time"], decimals=0)
                    .tab_spanner(
                        label="Metrics",
                        columns=["Recall", "Fit_Time"]
                    )
                    .cols_label(
                        Grid_Variable=html(""),
                        Parameters=html(f"{param.title()} <br> Parameter"),
                        Fit_Time=html("Fit Time (s)")
                    )
                )
            else:
                continue
            result_table[alt] = table

## Neurons

In [103]:
result_table['test_results_neurons-hidden_layer_sizes'].tab_header(
    title="Number of Neurons",
    subtitle="Test Set Prediction Results")

GT(_tbl_data=shape: (2, 6)
┌───────────────┬────────────┬──────────┬──────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64      ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪══════════╪══════════╪════════════╡
│ baseline      ┆ 100        ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832   │
│               ┆ 500        ┆ 0.448487 ┆ 0.729403 ┆ 0.79575  ┆ 379.136686 │
└───────────────┴────────────┴──────────┴──────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x169538f70>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Hidden_Layer_Sizes <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Number of Neurons', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695391e0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x169539a20>, <great_tables._gt_data.FormatInfo object at 0x169538340>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsIn

In [104]:
result_table['grid_results_neurons-hidden_layer_sizes'].tab_header(
    title="Number of Neurons",
    subtitle="Cross Validation Results")

GT(_tbl_data=shape: (5, 4)
┌───────────────┬────────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪════════════╡
│               ┆ 500        ┆ 0.928545 ┆ 379.136686 │
│               ┆ 250        ┆ 0.885997 ┆ 201.331052 │
│ baseline      ┆ 100        ┆ 0.820933 ┆ 67.24832   │
│               ┆ 50         ┆ 0.813431 ┆ 68.73278   │
│               ┆ 1          ┆ 0.795639 ┆ 13.57265   │
└───────────────┴────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x16953b7f0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Hidden_Layer_Sizes <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Number of Neurons', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1694f3cd0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1694f1f90>, <great_tables._gt_data.FormatInfo object at 0x1694f3790>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), tabl

## Layers

In [105]:
result_table['test_results_layers-hidden_layer_sizes'].tab_header(
    title="Number of Layers",
    subtitle="Test Set Prediction Results")

GT(_tbl_data=shape: (2, 6)
┌───────────────┬───────────────────────────┬──────────┬──────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters                ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time   │
│ ---           ┆ ---                       ┆ ---      ┆ ---      ┆ ---      ┆ ---        │
│ str           ┆ str                       ┆ f64      ┆ f64      ┆ f64      ┆ f64        │
╞═══════════════╪═══════════════════════════╪══════════╪══════════╪══════════╪════════════╡
│ baseline      ┆ 100                       ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832   │
│               ┆ [100, 100, 100, 100, 100] ┆ 0.518116 ┆ 0.760817 ┆ 0.809887 ┆ 832.888808 │
└───────────────┴───────────────────────────┴──────────┴──────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x169598760>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Hidden_Layer_Sizes <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Number of Layers', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695992d0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x169598550>, <great_tables._gt_data.FormatInfo object at 0x16953bd60>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), ta

In [106]:
result_table['grid_results_layers-hidden_layer_sizes'].tab_header(
    title="Number of Layers",
    subtitle="Cross Validation Results")

GT(_tbl_data=shape: (5, 4)
┌───────────────┬───────────────────────────────────┬──────────┬─────────────┐
│ Grid_Variable ┆ Parameters                        ┆ Recall   ┆ Fit_Time    │
│ ---           ┆ ---                               ┆ ---      ┆ ---         │
│ str           ┆ str                               ┆ f64      ┆ f64         │
╞═══════════════╪═══════════════════════════════════╪══════════╪═════════════╡
│               ┆ [100, 100, 100, 100, 100]         ┆ 0.972256 ┆ 832.888808  │
│               ┆ [100, 100, 100, 100, 100, 100, 1… ┆ 0.969954 ┆ 1083.540529 │
│               ┆ [100, 100, 100]                   ┆ 0.954148 ┆ 601.686462  │
│ baseline      ┆ 100                               ┆ 0.820933 ┆ 67.24832    │
│               ┆ [100]                             ┆ 0.820933 ┆ 111.931144  │
└───────────────┴───────────────────────────────────┴──────────┴─────────────┘, _body=<great_tables._gt_data.Body object at 0x169539f30>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Hidden_Layer_Sizes <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Number of Layers', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x16953b070>, _formats=[<great_tables._gt_data.FormatInfo object at 0x169538b20>, <great_tables._gt_data.FormatInfo object at 0x16953af50>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_sty

## Activation Function

In [107]:
result_table['test_results-activation'].tab_header(
    title="Activation Function",
    subtitle="Test Set Prediction Results")

GT(_tbl_data=shape: (2, 6)
┌───────────────┬────────────┬──────────┬──────────┬──────────┬──────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time │
│ ---           ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str           ┆ str        ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞═══════════════╪════════════╪══════════╪══════════╪══════════╪══════════╡
│ baseline      ┆ relu       ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832 │
│               ┆ logistic   ┆ 0.660207 ┆ 0.793378 ┆ 0.753255 ┆ 154.2702 │
└───────────────┴────────────┴──────────┴──────────┴──────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x1695bb3d0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Activation <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Activation Function', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695bb880>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695bb340>, <great_tables._gt_data.FormatInfo object at 0x1695bb040>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category

In [108]:
result_table['grid_results-activation'].tab_header(
    title="Activation Function",
    subtitle="Cross Validation Results")

GT(_tbl_data=shape: (4, 4)
┌───────────────┬────────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪════════════╡
│               ┆ logistic   ┆ 0.853467 ┆ 154.2702   │
│               ┆ tanh       ┆ 0.844433 ┆ 185.283824 │
│ baseline      ┆ relu       ┆ 0.820933 ┆ 67.24832   │
│               ┆ relu       ┆ 0.820933 ┆ 100.009219 │
└───────────────┴────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1695bae00>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Activation <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Activation Function', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695bbee0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695bbd90>, <great_tables._gt_data.FormatInfo object at 0x1695c01f0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, cat

## Alpha

In [109]:
result_table['test_results-alpha']

GT(_tbl_data=shape: (2, 6)
┌───────────────┬────────────┬──────────┬──────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64      ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪══════════╪══════════╪════════════╡
│ baseline      ┆ 0.0001     ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832   │
│               ┆ 0.0        ┆ 0.719418 ┆ 0.811282 ┆ 0.745786 ┆ 101.764314 │
└───────────────┴────────────┴──────────┴──────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1695b9b70>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Alpha <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Alpha', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695ba020>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695b9ae0>, <great_tables._gt_data.FormatInfo object at 0x1695b97e0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='t

In [110]:
result_table['grid_results-alpha']

GT(_tbl_data=shape: (6, 4)
┌───────────────┬────────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪════════════╡
│               ┆ 0.0        ┆ 0.825685 ┆ 101.764314 │
│ baseline      ┆ 0.0001     ┆ 0.820933 ┆ 67.24832   │
│               ┆ 0.25       ┆ 0.810478 ┆ 76.787527  │
│               ┆ 0.5        ┆ 0.807625 ┆ 50.072301  │
│               ┆ 0.75       ┆ 0.806063 ┆ 35.558084  │
│               ┆ 1.0        ┆ 0.802635 ┆ 36.327108  │
└───────────────┴────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1695ba4d0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Alpha <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Alpha', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695baa70>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695bab60>, <great_tables._gt_data.FormatInfo object at 0x1695bac80>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', val

## Batch Size

In [111]:
result_table['test_results-batch_size']

GT(_tbl_data=shape: (2, 6)
┌───────────────┬────────────┬──────────┬──────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64      ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪══════════╪══════════╪════════════╡
│ baseline      ┆ auto       ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832   │
│               ┆ 100        ┆ 0.736729 ┆ 0.815755 ┆ 0.74262  ┆ 150.978772 │
└───────────────┴────────────┴──────────┴──────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x16957ea40>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Batch_Size <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Batch Size', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x169562050>, _formats=[<great_tables._gt_data.FormatInfo object at 0x169562320>, <great_tables._gt_data.FormatInfo object at 0x164f416c0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, c

In [112]:
result_table['grid_results-batch_size']

GT(_tbl_data=shape: (5, 4)
┌───────────────┬────────────┬──────────┬─────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ Fit_Time    │
│ ---           ┆ ---        ┆ ---      ┆ ---         │
│ str           ┆ str        ┆ f64      ┆ f64         │
╞═══════════════╪════════════╪══════════╪═════════════╡
│               ┆ 100        ┆ 0.832282 ┆ 150.978772  │
│               ┆ 1000       ┆ 0.822254 ┆ 80.844334   │
│ baseline      ┆ auto       ┆ 0.820933 ┆ 67.24832    │
│               ┆ 500        ┆ 0.820736 ┆ 74.753082   │
│               ┆ 1          ┆ 0.806049 ┆ 1881.966173 │
└───────────────┴────────────┴──────────┴─────────────┘, _body=<great_tables._gt_data.Body object at 0x169598c70>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Batch_Size <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Batch Size', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x169599420>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695995d0>, <great_tables._gt_data.FormatInfo object at 0x169599720>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_bo

## Learning Rate

In [113]:
result_table['test_results-learning_rate']

GT(_tbl_data=shape: (2, 6)
┌───────────────┬────────────┬──────────┬──────────┬──────────┬───────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time  │
│ ---           ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ str           ┆ str        ┆ f64      ┆ f64      ┆ f64      ┆ f64       │
╞═══════════════╪════════════╪══════════╪══════════╪══════════╪═══════════╡
│ baseline      ┆ constant   ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832  │
│               ┆ constant   ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 96.904531 │
└───────────────┴────────────┴──────────┴──────────┴──────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x1695caad0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Learning_Rate <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Learning Rate', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695cafb0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695caa40>, <great_tables._gt_data.FormatInfo object at 0x1695ca740>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, cat

In [114]:
result_table['grid_results-learning_rate']

GT(_tbl_data=shape: (4, 4)
┌───────────────┬────────────┬──────────┬───────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ Fit_Time  │
│ ---           ┆ ---        ┆ ---      ┆ ---       │
│ str           ┆ str        ┆ f64      ┆ f64       │
╞═══════════════╪════════════╪══════════╪═══════════╡
│ baseline      ┆ constant   ┆ 0.820933 ┆ 67.24832  │
│               ┆ constant   ┆ 0.820933 ┆ 96.904531 │
│               ┆ invscaling ┆ 0.820933 ┆ 99.450832 │
│               ┆ adaptive   ┆ 0.820933 ┆ 97.082339 │
└───────────────┴────────────┴──────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x1695ca500>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Learning_Rate <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Learning Rate', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695cb940>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695cba30>, <great_tables._gt_data.FormatInfo object at 0x1695cbb50>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table'

## Initial Learning Rate

In [115]:
result_table['test_results-learning_rate_init'].tab_header(
    title="Initial Learning Rate",
    subtitle="Test Set Prediction Results")

GT(_tbl_data=shape: (2, 6)
┌───────────────┬────────────┬──────────┬──────────┬──────────┬───────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time  │
│ ---           ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ str           ┆ str        ┆ f64      ┆ f64      ┆ f64      ┆ f64       │
╞═══════════════╪════════════╪══════════╪══════════╪══════════╪═══════════╡
│               ┆ 0.01       ┆ 0.767905 ┆ 0.816309 ┆ 0.73001  ┆ 72.824502 │
│ baseline      ┆ 0.001      ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832  │
└───────────────┴────────────┴──────────┴──────────┴──────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x1695c22f0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Learning_Rate_Init <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Initial Learning Rate', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695c27a0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695c2260>, <great_tables._gt_data.FormatInfo object at 0x1695c1f60>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(s

In [116]:
result_table['grid_results-learning_rate_init'].tab_header(
    title="Initial Learning Rate",
    subtitle="Cross Validation Results")

GT(_tbl_data=shape: (4, 4)
┌───────────────┬────────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪════════════╡
│               ┆ 0.01       ┆ 0.85262  ┆ 72.824502  │
│ baseline      ┆ 0.001      ┆ 0.820933 ┆ 67.24832   │
│               ┆ 0.0001     ┆ 0.816108 ┆ 139.893551 │
│               ┆ 0.1        ┆ 0.801173 ┆ 39.018317  │
└───────────────┴────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1695c1d20>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Learning_Rate_Init <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Initial Learning Rate', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695c3130>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695c3220>, <great_tables._gt_data.FormatInfo object at 0x1695c3340>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss

## Max Iterations

In [125]:
result_table['test_results-max_iter'].tab_header(
    title="Maximum Iterations",
    subtitle="Test Set Prediction Results")

GT(_tbl_data=shape: (2, 6)
┌───────────────┬────────────┬──────────┬──────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64      ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪══════════╪══════════╪════════════╡
│ baseline      ┆ 200        ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832   │
│               ┆ 100        ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 105.687169 │
└───────────────┴────────────┴──────────┴──────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1695c3a90>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Max_Iter <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Maximum Iterations', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695c3d90>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695c3a00>, <great_tables._gt_data.FormatInfo object at 0x1695c2e60>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=T

In [126]:
result_table['grid_results-max_iter'].tab_header(
    title="Maximum Iterations",
    subtitle="Cross Validation Results")

GT(_tbl_data=shape: (5, 4)
┌───────────────┬────────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪════════════╡
│ baseline      ┆ 200        ┆ 0.820933 ┆ 67.24832   │
│               ┆ 100        ┆ 0.820933 ┆ 105.687169 │
│               ┆ 250        ┆ 0.820933 ┆ 106.422979 │
│               ┆ 500        ┆ 0.820933 ┆ 106.836757 │
│               ┆ 1000       ┆ 0.820933 ┆ 77.817537  │
└───────────────┴────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1695c8430>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Max_Iter <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Maximum Iterations', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695c8970>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695c8a60>, <great_tables._gt_data.FormatInfo object at 0x1695c8b80>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_

## Momentum

In [119]:
result_table['test_results-momentum']

GT(_tbl_data=shape: (2, 6)
┌───────────────┬────────────┬──────────┬──────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64      ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪══════════╪══════════╪════════════╡
│ baseline      ┆ 0.9        ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832   │
│               ┆ 0.0        ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 108.784148 │
└───────────────┴────────────┴──────────┴──────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1694f2dd0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Momentum <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Momentum', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695b87c0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695b82e0>, <great_tables._gt_data.FormatInfo object at 0x1695b8130>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, categ

In [120]:
result_table['grid_results-momentum']

GT(_tbl_data=shape: (6, 4)
┌───────────────┬────────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪════════════╡
│ baseline      ┆ 0.9        ┆ 0.820933 ┆ 67.24832   │
│               ┆ 0.0        ┆ 0.820933 ┆ 108.784148 │
│               ┆ 0.25       ┆ 0.820933 ┆ 109.241719 │
│               ┆ 0.5        ┆ 0.820933 ┆ 108.981126 │
│               ┆ 0.75       ┆ 0.820933 ┆ 86.033771  │
│               ┆ 1.0        ┆ 0.820933 ┆ 76.221695  │
└───────────────┴────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1695b8c70>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Momentum <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Momentum', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695b9210>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695b9300>, <great_tables._gt_data.FormatInfo object at 0x1695b9420>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px

## Stop After Number of Iterations of No Change 

In [121]:
result_table['test_results-n_iter_no_change'].tab_header(
    title="Number of Iterations of No Change",
    subtitle="Test Set Prediction Results")

GT(_tbl_data=shape: (2, 6)
┌───────────────┬────────────┬──────────┬──────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64      ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪══════════╪══════════╪════════════╡
│ baseline      ┆ 10         ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832   │
│               ┆ 100        ┆ 0.712601 ┆ 0.808466 ┆ 0.745294 ┆ 399.201779 │
└───────────────┴────────────┴──────────┴──────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1695c92d0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='N_Iter_No_Change <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Number of Iterations of No Change', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695c9780>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695c9240>, <great_tables._gt_data.FormatInfo object at 0x1695c8f40>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_c

In [122]:
result_table['grid_results-n_iter_no_change'].tab_header(
    title="Number of Iterations of No Change",
    subtitle="Cross Validation Results")

GT(_tbl_data=shape: (5, 4)
┌───────────────┬────────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪════════════╡
│               ┆ 100        ┆ 0.842093 ┆ 399.201779 │
│               ┆ 250        ┆ 0.842093 ┆ 400.506371 │
│               ┆ 500        ┆ 0.842093 ┆ 281.170787 │
│               ┆ 50         ┆ 0.841426 ┆ 356.611591 │
│ baseline      ┆ 10         ┆ 0.820933 ┆ 67.24832   │
└───────────────┴────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1695c9c30>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='N_Iter_No_Change <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Number of Iterations of No Change', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695ca170>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695ca260>, <great_tables._gt_data.FormatInfo object at 0x1695ca380>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#

## Solver

In [123]:
result_table['test_results-solver']

GT(_tbl_data=shape: (2, 6)
┌───────────────┬────────────┬──────────┬──────────┬──────────┬───────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time  │
│ ---           ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ str           ┆ str        ┆ f64      ┆ f64      ┆ f64      ┆ f64       │
╞═══════════════╪════════════╪══════════╪══════════╪══════════╪═══════════╡
│ baseline      ┆ adam       ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 67.24832  │
│               ┆ adam       ┆ 0.752738 ┆ 0.814254 ┆ 0.732085 ┆ 76.121285 │
└───────────────┴────────────┴──────────┴──────────┴──────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x1695c0bb0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Solver <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Solver', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695c1060>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695c0b20>, <great_tables._gt_data.FormatInfo object at 0x1695c0820>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table',

In [124]:
result_table['grid_results-solver']

GT(_tbl_data=shape: (3, 4)
┌───────────────┬────────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Recall   ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪══════════╪════════════╡
│ baseline      ┆ adam       ┆ 0.820933 ┆ 67.24832   │
│               ┆ adam       ┆ 0.820933 ┆ 76.121285  │
│               ┆ sgd        ┆ 0.811662 ┆ 148.127234 │
└───────────────┴────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x1695c0ee0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Solver <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Solver', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1695c1990>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1695c1a80>, <great_tables._gt_data.FormatInfo object at 0x1695c1ba0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bo